In [24]:
import requests
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

def search(search_type, filters='', fields = [], other_args = {}):
    base_URL = 'https://banks.data.fdic.gov/api/'
    finished = False
    data = []
    
    params = {'limit' : 500}
    params['filters'] = filters
    params['offset'] = 0
    params['fields'] = ','.join(fields)
    
    for key, value in other_args.items():
        params[key] = value
    
    while not finished:
        try:
            r = requests.get(base_URL + search_type, params=params).json()
        except ConnectionError:
            print("Connection error")
            continue
        data.extend(r['data'])
        if len(r['data']) < params['limit']:
            finished = True
        params['offset'] += params['limit']
        
    return [x['data'] for x in data]

In [42]:
r = search('financials', 'CALLYM:202103')

In [45]:
data = pd.DataFrame(r)

In [49]:
data.head()

,DRRENROW,EINTXQ,ASCERES,EXTRA,DEPUSBKF,UYANMTG,DRRENROT,RBCT1CER,RTPOC,LNOT5T15,...,P3RSLNFM,P9RENROW,LNMUNI,LNOT1T3,P9RENROT,SZDRAUTO,IINSUNDQ,CHBALNI,LNRECNFM,ID
0,0.0,163.0,NaN,0.0,NaN,None,0.0,NaN,NaN,8022.0,...,0.0,0.0,1124.0,14197.0,0.0,NaN,NaN,1746.0,881.0,10004_20210331
1,0.0,369.0,NaN,0.0,NaN,None,0.0,11.60,NaN,41483.0,...,0.0,0.0,3511.0,41382.0,0.0,NaN,NaN,3420.0,2419.0,10011_20210331
2,0.0,96.0,NaN,0.0,NaN,None,0.0,16.28,NaN,9148.0,...,0.0,0.0,2860.0,37698.0,0.0,NaN,NaN,3297.0,1576.0,10012_20210331
3,0.0,250.0,NaN,0.0,NaN,None,0.0,NaN,NaN,7974.0,...,0.0,0.0,234.0,39003.0,0.0,NaN,NaN,7540.0,2422.0,10015_20210331
4,0.0,577.0,NaN,0.0,NaN,None,0.0,NaN,NaN,59983.0,...,0.0,0.0,36582.0,353354.0,0.0,NaN,NaN,23216.0,16274.0,10044_20210331


In [52]:
nas = []
for col in data.columns:
    count = 0
    for num in data[col]:
        if pd.isna(num):
            count += 1
    if count > 0:
        nas.append((col, count))

In [55]:
nas_to_drop = []
for n in nas:
    if n[1] > 17:
        nas_to_drop.append(n[0])

In [68]:
nas_to_drop

455

In [63]:
new_data = data.drop(nas_to_drop + ['ID'], axis =1)

In [64]:
new_data.shape[0] - new_data.dropna().shape[0]

18

In [65]:
cdata = new_data.dropna()

In [74]:
cdataf = cdata.select_dtypes(exclude=['object', 'string'])

{dtype('int64'), dtype('float64')}